In [1]:
from setup import init_django

In [2]:
init_django()

In [3]:
from market.models import StockQuote

In [4]:
from django.db.models import Avg, F, RowRange, Window, Max, Min
from django.db.models.functions import TruncDate, FirstValue
from django.utils import timezone
from datetime import timedelta
from decimal import Decimal

In [5]:
days_ago = 365
now = timezone.now()
start_date = now - timedelta(days=days_ago)
end_date = now
ticker = "X:BTCUSD"


In [6]:
latest_daily_timestamps = (
    StockQuote.timescale.filter(company__ticker = ticker, time__range=(start_date, end_date))
    .time_bucket('time', '1 day')
    .annotate(date=TruncDate('time'))
    .values('company', 'date')
    .annotate(latest_time=Max('time'))
    .values('company', 'date', 'latest_time')
    .order_by('date')
)
actual_timestamps = list(set([x['latest_time'] for x in latest_daily_timestamps]))
actual_timestamps

[datetime.datetime(2025, 8, 1, 23, 59, tzinfo=datetime.timezone.utc),
 datetime.datetime(2024, 8, 23, 23, 59, tzinfo=datetime.timezone.utc),
 datetime.datetime(2025, 4, 29, 23, 59, tzinfo=datetime.timezone.utc),
 datetime.datetime(2025, 4, 1, 23, 59, tzinfo=datetime.timezone.utc),
 datetime.datetime(2025, 1, 31, 23, 59, tzinfo=datetime.timezone.utc),
 datetime.datetime(2025, 1, 3, 23, 59, tzinfo=datetime.timezone.utc),
 datetime.datetime(2025, 5, 7, 23, 59, tzinfo=datetime.timezone.utc),
 datetime.datetime(2025, 6, 9, 23, 59, tzinfo=datetime.timezone.utc),
 datetime.datetime(2024, 12, 6, 23, 59, tzinfo=datetime.timezone.utc),
 datetime.datetime(2025, 7, 27, 23, 59, tzinfo=datetime.timezone.utc),
 datetime.datetime(2024, 11, 8, 23, 59, tzinfo=datetime.timezone.utc),
 datetime.datetime(2025, 3, 13, 23, 59, tzinfo=datetime.timezone.utc),
 datetime.datetime(2025, 7, 15, 23, 59, tzinfo=datetime.timezone.utc),
 datetime.datetime(2024, 10, 16, 23, 59, tzinfo=datetime.timezone.utc),
 datetime.

In [7]:
len(actual_timestamps)

365

In [8]:
qs = StockQuote.objects.filter(
    company__ticker=ticker, 
    time__range=(start_date, end_date),
    #time__in=actual_timestamps
)

In [9]:
len(qs)

524278

In [10]:
def get_volume_trend(queryset, days=5):
    """
    Calculate the volume trend for a given queryset of StockQuote objects.

    Args:
    queryset (QuerySet): A queryset of Stock Quote objects.
    days (int): The number of days to consider.

    Returns:
    dict: A dictionary containing the average volume, latest volume and percentage volume change.
    """
    start = -(days - 1)
    data = qs.annotate(
        avg_volume=Window(
            expression=Avg('volume'),
            order_by=F('time').asc(),
            partition_by=[],
            frame=RowRange(start=start, end=0),
        )
    ).order_by('-time').first()

    if not data:
        return None
    volume_change = ((data.volume - data.avg_volume) / data.avg_volume) * 100

    return {
        'avg_volume': float(data.avg_volume),
        'latest_volume': float(data.volume),
        'volume_change_percent': float(volume_change),
    }

In [11]:
get_volume_trend(queryset=qs, days=10)


{'avg_volume': 2.34215,
 'latest_volume': 3.377,
 'volume_change_percent': 44.18376278205922}

In [12]:
def get_simple_target(ticker, timestamps=[], days=180):
    """
    Simplified price target calculation
    """
    end_date = timezone.now()
    start_date = end_date - timedelta(days=days)
    lookups = {
        "company__ticker": ticker,
        "time__range": (start_date, end_date)
    }
    if len(timestamps) > 0:
        lookups['time__in'] = timestamps
    daily_data = (
        StockQuote.timescale
        .filter(**lookups)
        .time_bucket('time', '1 day')
        .annotate(
            latest_price=Window(
                expression=FirstValue('close_price'),
                partition_by=[],
                order_by=F('time').desc()
            )
        )
        .aggregate(
            current_price=Max('latest_price'),
            avg_price=Avg('close_price'),
            highest=Max('high_price'),
            lowest=Min('low_price')
        )
    )
    
    if not daily_data:
        return None
        
    current_price = float(daily_data['current_price'])
    avg_price = float(daily_data['avg_price'])
    price_range = float(daily_data['highest']) - float(daily_data['lowest'])
    
    # Simple target based on average price and recent range
    conservative_target = current_price + (price_range * 0.382)  # 38.2% Fibonacci
    aggressive_target = current_price + (price_range * 0.618)   # 61.8% Fibonacci
    
    return {
        'current_price': current_price,
        'conservative_target': conservative_target,
        'aggressive_target': aggressive_target,
        'average_price': avg_price
    }

In [13]:
get_simple_target(ticker=ticker)

{'current_price': 115028.83,
 'conservative_target': 133678.82254,
 'aggressive_target': 145200.80746,
 'average_price': 99126.7087078408}